In [21]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

##GPU 사용 시
device = torch.device("cuda:0")

In [22]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [23]:
RANDOM_SEED = 17

In [24]:
dataset = pd.read_csv("data/train_data.csv",index_col=False)
# rd_augmentation = pd.read_csv("data/train_rd_augmentation.csv",index_col=False)
# rs_augmentation = pd.read_csv("data/train_rs_augmentation.csv",index_col=False)
test = pd.read_csv("data/test_data.csv")

In [25]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_key]]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int32(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int32(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))

using cached model


In [29]:
## Setting parameters
max_len = 40
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
NUM_CLASS = 7

epochs_no_improve = 0
min_val_loss = np.Inf
n_epochs_stop = 1

data_train = BERTDataset(dataset, "title", "topic_idx", tok, max_len, True, False)
data_test = BERTDataset(test, "title", None, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_CLASS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [31]:
# 계층 교차 검증
n_fold = 5  
seed = 42

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test.shape[0], 7))

for i, (i_trn, i_val) in enumerate(cv.split(train_dataloader), 1):
    print(f'training model for CV #{i}')
    
    model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
    
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    t_total = len(train_dataloader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)

    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
    
    for e in range(num_epochs):
        train_acc = 0.0
        test_acc = 0.0
        
        model.train()
        
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader[i_trn])):
            optimizer.zero_grad()

            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length

            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()

            scheduler.step()  # Update learning rate schedule
            
            train_acc += calc_accuracy(out, label)
            
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        
        print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

        model.eval()
        
        val_loss = 0
        
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader[i_val])):

            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out,label)

            val_loss += loss.data.cpu().numpy()
            test_acc += calc_accuracy(out, label)

        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
        print("epoch {} val_loss {}".format(e+1, val_loss / (batch_id+1)))

        if val_loss < min_val_loss:
            torch.save(model,"models/kfold-kobert-1.pth")
            epochs_no_improve = 0
            min_val_loss = val_loss
        else :
            epochs_no_improve += 1

        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!' )
            early_stop = True
            break
        else:
            print("Keep going!")
            continue

        if early_stop:
            print("Stopped")
            break
            
    model = torch.load("models/kfold-kobert-1.pth")
    model.to(device)
    model.eval()
    
    pred = []
    outs = []

    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        out = model(token_ids, valid_length, segment_ids)
        
        

        _,max_indices = torch.max(out,1)
        for idx in max_indices.cpu().numpy():
            pred.append(idx)

        for o in out.detach():
            outs.append(o)
    
    test_y += outs / n_fold

TypeError: split() missing 1 required positional argument: 'y'

In [5]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

,index,title,topic_idx
25339,25339,더민주 서영교 여파 지역위원장 심사기준 강화,6
24704,24704,맛집에 너그러운 한국인 해외여행서도 JMT 찾았다,3
1834,1834,특징주 삼성물산 지배구조 이슈 부각에 강세종합,1
17604,17604,생필품난 베네수엘라 콜롬비아와의 국경 1년 만에 재개방,4
19362,19362,금태섭 국민 10명 중 8명 판결문 공개 원해,6
